In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
import random 
pd.set_option('display.max_columns', 5000)

In [2]:
prepend = '' #+ 'NE_'
combined_data = pd.read_csv('../data/processed/' + prepend + 'multi_var_unscaled.csv',  index_col=0)
print(combined_data.shape)
combined_data.head()

(786, 32)


,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,NursingCt,HospCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003.0,0.154926,0.213197,0.163596,0.154089,0.163596,0.230107,0.206183,0.230107,114.6,3.0,9.7,31.3,3.6,115.5,0.204433,2.57,0.014058,0.006646,0.106659,0.051412,0.029827,218022.0,0.000115,0.001523,0.046468,0.089404,66.0,-33.0,0,11,6.0,0.000028
1015.0,0.144609,0.213616,0.150983,0.144994,0.150983,0.227703,0.214584,0.227703,195.7,3.0,15.9,18.0,4.7,91.0,0.177175,2.51,0.013677,0.012986,0.098217,0.043508,0.060292,114277.0,0.000149,0.004830,0.039107,0.211355,63.0,NaN,0,11,3.0,0.000026
1049.0,0.137749,0.213115,0.148673,0.146733,0.148673,0.247374,0.239709,0.247374,91.5,6.0,25.9,12.7,3.8,79.2,0.172613,2.66,0.005407,0.009806,0.116453,0.060993,0.023170,71385.0,0.000098,0.001877,0.148421,0.018561,70.7,NaN,0,40,5.0,0.000070
1051.0,0.150897,0.258393,0.164240,0.159221,0.164240,0.306401,0.289678,0.306401,128.2,2.0,13.6,24.5,3.4,121.0,0.151428,2.57,0.012908,0.006668,0.090405,0.062623,0.049116,81887.0,0.000085,0.001685,0.030127,0.214564,75.0,-45.0,0,25,7.0,0.000085
1055.0,0.175885,0.257832,0.150136,0.152809,0.150136,0.206914,0.211104,0.206914,195.2,3.0,15.5,17.7,4.1,90.0,0.190115,2.62,0.010400,0.012673,0.099658,0.045151,0.039209,102501.0,0.000146,0.006117,0.040156,0.157013,62.8,NaN,5,0,8.0,0.000078


In [3]:
adjacency = pd.read_excel("../data/raw/adjacent_counties.xlsx")
adjacency.head()

,County_1_Name,County_1_FIPS,County_2_Name,County_2_FIPS
0,"Autauga County, AL",1001,"Autauga County, AL",1001
1,"Autauga County, AL",1001,"Chilton County, AL",1021
2,"Autauga County, AL",1001,"Dallas County, AL",1047
3,"Autauga County, AL",1001,"Elmore County, AL",1051
4,"Autauga County, AL",1001,"Lowndes County, AL",1085


In [4]:
colnames = ['FIPS1', 'FIPS2']
for col in combined_data.columns: 
    colnames.append(col)
print(colnames)

['FIPS1', 'FIPS2', '1 Month Prior Intra-Mobility', '1 Month Prior Inter-Mobility', '2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility', '2wk Post Intra-Mobility', '2wk Prior Inter-Mobility', '2wk Onset Inter-Mobility', '2wk Post Inter-Mobility', 'Density per square mile of land area - Population', 'Rural-urban_Continuum Code_2013', 'Percent of adults with less than a high school diploma 2014-18', "Percent of adults with a bachelor's degree or higher 2014-18", 'Unemployment_rate_2018', 'Med_HH_Income_Percent_of_State_Total_2018', 'Total_age65plus', 'Total households!!Average household size', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)', 'SCHOOL ENROLLMENT!!Popu

In [5]:
pairwise_data = pd.DataFrame(columns=colnames)
pairwise_data.head()

,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,NursingCt,HospCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths


In [6]:
count = 0 
for index,row in adjacency.iterrows():
    fips1 = row['County_1_FIPS']
    fips2 = row['County_2_FIPS']
    if fips1 >= fips2:  # To avoid duplicates 
        continue
    if fips1 not in combined_data.index or fips2 not in combined_data.index:
        continue
    add_row = {}
    if random.randint(0,1) == 0:    
        add_row['FIPS1'] = fips1
        add_row['FIPS2'] = fips2
    else: 
        add_row['FIPS1'] = fips2
        add_row['FIPS2'] = fips1
    # Take differences 
    found_na = False
    for col in combined_data.columns:
        add_row[col] = combined_data.loc[fips1, col] - combined_data.loc[fips2,col]
        if (np.isnan(combined_data.loc[fips1, col]) or np.isnan(combined_data.loc[fips2,col])) and not found_na:
            print(add_row[col])
            count +=1
            found_na = True 
    pairwise_data = pairwise_data.append(add_row, ignore_index=True)
#     if index % 20 == 0:
#         print(pairwise_data.head())
print(count)
pairwise_data.head()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,NursingCt,HospCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
0,1003.0,1097.0,-0.006221,0.006600,0.026400,0.019504,0.026400,0.065852,0.041633,0.065852,-221.3,1.0,-4.1,8.4,-1.1,28.1,0.042404,0.00,-0.002601,-0.005250,0.002395,-0.002624,-0.020843,-195735.0,0.000011,-0.000894,0.015899,-0.271506,0.1,-18.0,-8.0,11.0,-33.0,-0.000067
1,12033.0,1003.0,0.023561,0.026387,0.048169,0.034956,0.048169,0.078220,0.051720,0.078220,-338.8,1.0,0.5,4.9,0.1,24.1,0.035995,0.06,-0.000302,-0.007584,0.016799,0.001617,-0.043407,-97512.0,-0.000009,-0.001646,-0.011729,-0.142676,0.1,-9.0,0.0,11.0,-5.0,-0.000007
2,1055.0,1015.0,-0.031277,-0.044216,0.000847,-0.007815,0.000847,0.020789,0.003481,0.020789,0.5,0.0,0.4,0.3,0.6,1.0,-0.012940,-0.11,0.003277,0.000313,-0.001440,-0.001642,0.021083,11776.0,0.000002,-0.001287,-0.001049,0.054342,0.2,NaN,-5.0,11.0,-5.0,-0.000052
3,1081.0,1017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.0,0.0,NaN,NaN
4,1017.0,13285.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,0.0,NaN,NaN


In [7]:
pairwise_data.shape

(1095, 34)

In [8]:
pre_partial_corr = pairwise_data.filter(["Rural-urban_Continuum Code_2013", "2wk Prior Intra-Mobility", "Total_age65plus", "HospCt", "NursingCt", "normalized_deaths"])
pre_partial_corr = pre_partial_corr.dropna()
pre_partial_corr.to_csv('../data/processed/' + prepend + 'local_pairwise_multi_var_unscaled.csv', index=False)
print(pre_partial_corr.shape)
pre_partial_corr.head()

(811, 6)


,Rural-urban_Continuum Code_2013,2wk Prior Intra-Mobility,Total_age65plus,HospCt,NursingCt,normalized_deaths
0,1.0,0.026400,0.042404,-0.000894,0.000011,-0.000067
1,1.0,0.048169,0.035995,-0.001646,-0.000009,-0.000007
2,0.0,0.000847,-0.012940,-0.001287,0.000002,-0.000052
5,3.0,-0.001463,-0.017502,-0.004240,-0.000048,-0.000008
6,2.0,0.007360,0.002234,-0.000620,0.000004,-0.000013


In [9]:
pairwise_data = pairwise_data.dropna()
pairwise_data.shape

(583, 34)

## Standard Deviation Scaling

In [10]:
scaler = StandardScaler() 
print(pairwise_data.to_numpy().shape)
# scaled_data = scaler.fit_transform(combined_data)
for i in range(2, len(pairwise_data.columns.values)- 2 ): # 1 to -2 so that it scales all except FIPs, deaths, and normalized deaths 
    col = pairwise_data.columns.values[i]
#     combined_data[col] = scaled_data[:, i]
    pairwise_data[col] = scaler.fit_transform(np.array(pairwise_data[col]).reshape(-1,1)).reshape(-1)

(583, 34)


In [11]:
print(pairwise_data.max(axis=0))
print(pairwise_data.min(axis=0))
pairwise_data.head()

FIPS1                                                                                                55133.000000
FIPS2                                                                                                55133.000000
1 Month Prior Intra-Mobility                                                                             2.975976
1 Month Prior Inter-Mobility                                                                             3.213658
2wk Prior Intra-Mobility                                                                                 3.108538
2wk Onset Intra-Mobility                                                                                 3.143147
2wk Post Intra-Mobility                                                                                  3.108538
2wk Prior Inter-Mobility                                                                                 5.265570
2wk Onset Inter-Mobility                                                                

,FIPS1,FIPS2,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Total_age65plus,Total households!!Average household size,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,NursingCt,HospCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,Deaths,normalized_deaths
0,1003.0,1097.0,-0.342697,0.204549,1.251452,0.907112,1.251452,2.145553,1.423420,2.145553,0.027059,1.068392,-0.757302,0.647714,-0.794326,0.992035,0.793988,0.059330,-0.649475,-1.226761,0.169463,-0.202892,-0.539651,-0.231387,0.113579,-0.727074,0.237995,-2.055380,-0.057589,-1.367917,-1.050986,1.294704,-33.0,-0.000067
1,12033.0,1003.0,1.072502,0.904665,2.335637,1.674980,2.335637,2.562506,1.785262,2.562506,-0.001238,1.068392,0.126827,0.378963,0.022775,0.859024,0.674572,0.276417,-0.136818,-1.767844,1.053969,0.215399,-1.135186,-0.146080,-0.173696,-1.339735,-0.018766,-1.078738,-0.057589,-0.681604,0.021852,1.294704,-5.0,-0.000007
7,1051.0,1101.0,1.674893,2.880798,3.053564,2.699576,3.053564,5.265570,5.002506,5.265570,0.040786,-0.001836,-0.123036,-0.580859,-0.453867,0.772567,0.004968,0.312598,0.613842,-1.564061,-0.762023,0.621988,-0.412909,-0.186347,-0.670690,-2.046079,0.043495,-2.844225,1.178964,-1.825459,0.021852,2.839259,1.0,0.000059
10,1125.0,1073.0,0.551483,1.107270,-1.651463,-1.604922,-1.651463,-1.794419,-1.643833,-1.794419,0.187570,-2.142291,-0.219137,0.163963,-0.045317,0.220574,0.484115,-0.555751,0.243573,0.239223,0.465290,0.354604,-1.961163,0.329769,-0.259008,-1.831544,0.099200,0.865778,-1.456517,2.292418,1.094691,-3.101339,19.0,-0.000010
11,1081.0,13215.0,2.049380,3.173286,1.368273,1.296207,1.368273,1.461298,1.808261,1.461298,-0.075388,1.068392,-0.584320,0.609321,-1.066693,0.676135,-0.277584,0.348780,-0.422970,-1.027793,-0.779063,-0.123655,2.247777,-0.087637,-0.130255,-2.633390,-0.272957,-1.870864,-0.170003,0.385994,0.826481,-0.131040,15.0,0.000098


In [12]:
pairwise_data.to_csv('../data/processed/' + prepend + 'local_pairwise_multi_var_scaled.csv', index=False)